In [95]:
import glob
import pandas as pd
import numpy as np

import isodate

In [255]:
#config

COUNTRY_CODE = 'US'

In [256]:
# combine new datasets at first
folder_contents = glob.glob(f"C:\\Users\\George\\Desktop\\coding-ground\\Trending-YouTube-Scraper\\output\\*{COUNTRY_CODE}_videos.csv")

new_df = pd.DataFrame()

for file in folder_contents:
    print("checking", file)
    new_df = pd.concat([new_df, pd.read_csv(file)])

# align new df to the same format to the old one
# new_df['video_error_or_removed'] = False
new_df = new_df.rename(columns = {'categoryId':'category_id', 
                         'publishedAt':'publish_time', 
                         'channelTitle':'channel_title',
                         'view_count': 'views',
                         'channelId': 'channel_id'}) 

checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.01.11_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.02.11_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.03.11_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.04.11_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.15.10_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.16.10_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.17.10_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.18.10_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.19.10_US_videos.csv
checking C:\Users\George\Desktop\coding-ground\Trending-YouTube-Scraper\output\19.

In [257]:
# read old dataset
old_df = pd.read_csv(f"query_result/{COUNTRY_CODE}videos.csv")

In [258]:
# combine new dataset to the old one
updated_df = pd.concat([old_df, new_df], sort=False)

In [259]:
# before saving them, let's reformt some of the columns

# convert trending_date 18.31.11 -> 2018-11-31
def convert_trending_date(df):
    df['trending_date'] = pd.to_datetime(df.trending_date, format='%y.%d.%m')
    return df

# convert duration to seconds
# answer credit: https://stackoverflow.com/a/40432909
def change_duration(df):
    df1 = df[df.duration.notnull()]
    df2 = df[df.duration.isnull()]
    
    df1['duration'] = df1['duration'].apply(isodate.parse_duration)
    df1['duration'] = round(df1['duration']/np.timedelta64(1, 's'))
    return pd.concat([df1, df2])

## strip away quotation marks from tags
def strip_quotation(string):    
    temp = [x.strip('"') for x in string.split('|')]
    return "|".join(temp)    

# format tag column
def format_tags(df):
    
    df = df.replace("[none]", np.nan)
    
    df1 = df[df.tags.isnull()]
    df2 = df[df.tags.notnull()]
    df2['tags'] = df2['tags'].apply(strip_quotation)
    
    return pd.concat([df1, df2])

In [260]:
updated_df = change_duration(updated_df)
updated_df = convert_trending_date(updated_df)
updated_df = format_tags(updated_df)

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

In [261]:
# write new file
updated_df.to_csv(f"final_dataset/{COUNTRY_CODE}videos.csv", encoding='utf-8', index=False)